# Best optimizer parameters:

## RSI_STOCH:

### SELL ~ 
- RSI: low/high bound: **30/70**, timeperiod: **14**
- STOCH: low/high bound: **15/85**, fastk: **9**, slowk: **4**, slowd: **7**

# Parameter optimization

In [ ]:
import sys
import pandas as pd
import numpy as np
from optimizer import Optimizer
from os import environ


# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

# Get configs
configs = ConfigFactory.factory(environ).configs


pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI']
opt_limit = 100
load = False

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [25, 30, 35]},
              'STOCH': {'fastk_period': [5, 9, 14], 'slowk_period': [2, 3, 4],
                        'slowd_period': [3, 5, 7], 'low_bound': [10, 15, 20]}}

opt = Optimizer(pattern, optim_dict, **configs)
opt.clean_prev_stat()
rs = opt.optimize(pattern, ttype, opt_limit, load)

stat = pd.read_pickle(f'opt_{"_".join(pattern)}_{ttype}.pkl')
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat = stat.sort_values(['pct_right_forecast_avg', 'forecasts_num'], ascending=False)
stat[stat['forecasts_num'] > 10].head(20)

Number of combinations is 729


  0%|          | 0/729 [00:00<?, ?it/s]